<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/%22Hacker's_Guide_GPT_Incident_Analysis_Pretraining%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --upgrade gspread
!pip install oauth2client


In [7]:
from google.colab import auth
import gspread
from google.auth import default # Import default from google.auth instead of GoogleCredentials

# Authentication ke liye Google Colab ko authorize karein
auth.authenticate_user()

# Gspread ko authorize karein taa ke Google Sheet ko access kar sakein
creds, _ = default() # Get credentials using default() from google.auth
gc = gspread.authorize(creds) # Authorize gspread using the obtained credentials

In [9]:
# Google Sheet ko open karein (URL se)
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1O5Tj58XgGhZ-XuKlmO09iNA5fvGwA3RBG8CZC324azY/edit?usp=sharing')
sheet = worksheet.get_worksheet(0)  # Yahan 0 ka matlab hai pehla sheet


In [10]:
import pandas as pd

# Google Sheet se sara data get karein aur pandas dataframe mein store karein
data = sheet.get_all_records()
df = pd.DataFrame(data)

# Data ko dekhne ke liye print karein
print(df.head())


     Source IP Destination IP  Port            Event Type  \
0  192.168.1.5      10.0.0.20    22  Failed Login Attempt   

                        Reason/Analysis                       Remediation  
0  Brute-force attack attempt detected.  Block suspicious IP, enable MFA.  


In [11]:
!pip install transformers
!pip install torch


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# GPT-2 ko Hugging Face se load karein
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [15]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# GPT-2 ko Hugging Face se load karein
model_name = "gpt2"

In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# GPT-2 ko Hugging Face se load karein
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token  # Using eos_token as pad_token

model = GPT2LMHeadModel.from_pretrained(model_name)

# Event Type aur Remediation ko combine kar ke text bana rahe hain
df['text'] = df['Event Type'] + " " + df['Reason/Analysis'] + " " + df['Remediation']

# Dataset ko tokenize karein
def tokenize_function(example):
    return tokenizer(example, return_tensors='pt', truncation=True, padding='max_length')

# Tokenize texts
tokenized_texts = df['text'].apply(tokenize_function)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
from transformers import Trainer, TrainingArguments

# Training arguments define karein
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
)

# Update tokenize_function to include labels
def tokenize_function(example):
    tokenized = tokenizer(example, return_tensors='pt', truncation=True, padding='max_length')
    # The labels are the same as the input_ids shifted to the right by one token.
    tokenized['labels'] = tokenized['input_ids'].clone()
    return tokenized

# Tokenize texts with labels
tokenized_texts = df['text'].apply(tokenize_function).tolist() # Convert to a list of dictionaries

# Trainer define karein jo model ko fine-tune karega
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_texts,
)

# Model ko fine-tune karein
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=7.320645650227864, metrics={'train_runtime': 94.4487, 'train_samples_per_second': 0.032, 'train_steps_per_second': 0.032, 'total_flos': 1567752192000.0, 'train_loss': 7.320645650227864, 'epoch': 3.0})

In [20]:
model.save_pretrained("./fine_tuned_gpt")
tokenizer.save_pretrained("./fine_tuned_gpt")


('./fine_tuned_gpt/tokenizer_config.json',
 './fine_tuned_gpt/special_tokens_map.json',
 './fine_tuned_gpt/vocab.json',
 './fine_tuned_gpt/merges.txt',
 './fine_tuned_gpt/added_tokens.json')

In [41]:
import requests
from google.colab import userdata


In [43]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY_ONE')


In [44]:
import requests

def call_gemini_api(input_text):
    gemini_endpoint = "https://api.gemini.com/v1/analyze"  # Gemini API endpoint

    # Headers mein API key add karein
    headers = {
        "Authorization": f"Bearer {gemini_api_key}",
        "Content-Type": "application/json"
    }
    # Input text ko payload mein set karein
    payload = {"input_text": input_text}

    # Gemini API ko request bhejein
    response = requests.post(gemini_endpoint, headers=headers, json=payload)

    # Response check karein
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        return {"error": f"API request failed with status code {response.status_code}"}


In [45]:
import pandas as pd

# Google Sheets se data fetch karein
data = sheet.get_all_records()  # Google Sheets se data fetch karein
df = pd.DataFrame(data)

# Har row ko Gemini API ke sath analyze karein
for index, row in df.iterrows():
    input_text = f"Source IP: {row['Source IP']}, Destination IP: {row['Destination IP']}, Port: {row['Port']}, Event Type: {row['Event Type']}"
    analysis_result = call_gemini_api(input_text)
    print(f"Analysis for Row {index + 1}: {analysis_result}")


Analysis for Row 1: {'error': 'API request failed with status code 404'}


In [46]:
# Agar user input deta hai jisme security incident ki details hain (jaise Source IP, Destination IP, Port, ya Event Type), toh Gemini API ko call karke analysis karein.
# You can add your code to process user input and call Gemini API here.

# Example:
# user_input = "Source IP: 192.168.1.1, Destination IP: 8.8.8.8, Port: 80, Event Type: Login Attempt"

# def analyze_security_incident(user_input):
#    # Code to process user input and extract relevant details
#    # Call Gemini API to analyze the details

# analyze_security_incident(user_input)

SyntaxError: invalid syntax (<ipython-input-46-d88d912c3be3>, line 1)

In [47]:
# Endpoint URL: https://api.gemini.com/v1/analyze
# Headers:
#   Authorization: Bearer YOUR_GEMINI_API_KEY
# This is not executable Python code.
# It appears to be documentation or a configuration snippet.
# If you want to make a request using Python, you can use the requests library
# Here's an example:
# import requests
# headers = {
#     'Authorization': 'Bearer YOUR_GEMINI_API_KEY'
# }
# response = requests.get('https://api.gemini.com/v1/analyze', headers=headers)
  Content-Type: application/json


SyntaxError: invalid syntax (<ipython-input-47-e2a3ea6e1270>, line 1)

In [48]:
# prompt: PLZ SOLVE ERROR OF LAST PROMPT

from google.colab import auth
import gspread
from google.auth import default # Import default from google.auth instead of GoogleCredentials
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import requests
from google.colab import userdata
!pip install --upgrade gspread
!pip install oauth2client


# Authentication ke liye Google Colab ko authorize karein
auth.authenticate_user()

# Gspread ko authorize karein taa ke Google Sheet ko access kar sakein
creds, _ = default() # Get credentials using default() from google.auth
gc = gspread.authorize(creds) # Authorize gspread using the obtained credentials
# Google Sheet ko open karein (URL se)
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1O5Tj58XgGhZ-XuKlmO09iNA5fvGwA3RBG8CZC324azY/edit?usp=sharing')
sheet = worksheet.get_worksheet(0)  # Yahan 0 ka matlab hai pehla sheet


# Google Sheet se sara data get karein aur pandas dataframe mein store karein
data = sheet.get_all_records()
df = pd.DataFrame(data)

# Data ko dekhne ke liye print karein
print(df.head())

!pip install transformers
!pip install torch


# GPT-2 ko Hugging Face se load karein
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


# GPT-2 ko Hugging Face se load karein
model_name = "gpt2"

# GPT-2 ko Hugging Face se load karein
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token  # Using eos_token as pad_token

model = GPT2LMHeadModel.from_pretrained(model_name)

# Event Type aur Remediation ko combine kar ke text bana rahe hain
df['text'] = df['Event Type'] + " " + df['Reason/Analysis'] + " " + df['Remediation']

# Dataset ko tokenize karein
def tokenize_function(example):
    return tokenizer(example, return_tensors='pt', truncation=True, padding='max_length')

# Tokenize texts
tokenized_texts = df['text'].apply(tokenize_function)

# Training arguments define karein
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir='./logs',
)

# Update tokenize_function to include labels
def tokenize_function(example):
    tokenized = tokenizer(example, return_tensors='pt', truncation=True, padding='max_length')
    # The labels are the same as the input_ids shifted to the right by one token.
    tokenized['labels'] = tokenized['input_ids'].clone()
    return tokenized

# Tokenize texts with labels
tokenized_texts = df['text'].apply(tokenize_function).tolist() # Convert to a list of dictionaries

# Trainer define karein jo model ko fine-tune karega
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_texts,
)

# Model ko fine-tune karein
trainer.train()
model.save_pretrained("./fine_tuned_gpt")
tokenizer.save_pretrained("./fine_tuned_gpt")

gemini_api_key = userdata.get('GEMINI_API_KEY_ONE')


def call_gemini_api(input_text):
    gemini_endpoint = "https://api.gemini.com/v1/analyze"  # Gemini API endpoint

    # Headers mein API key add karein
    headers = {
        "Authorization": f"Bearer {gemini_api_key}",
        "Content-Type": "application/json"
    }
    # Input text ko payload mein set karein
    payload = {"input_text": input_text}

    # Gemini API ko request bhejein
    response = requests.post(gemini_endpoint, headers=headers, json=payload)

    # Response check karein
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print(f"API request failed with status code {response.status_code}")
        return {"error": f"API request failed with status code {response.status_code}"}



# Google Sheets se data fetch karein
data = sheet.get_all_records()  # Google Sheets se data fetch karein
df = pd.DataFrame(data)

# Har row ko Gemini API ke sath analyze karein
for index, row in df.iterrows():
    input_text = f"Source IP: {row['Source IP']}, Destination IP: {row['Destination IP']}, Port: {row['Port']}, Event Type: {row['Event Type']}"
    analysis_result = call_gemini_api(input_text)
    print(f"Analysis for Row {index + 1}: {analysis_result}")


     Source IP Destination IP  Port            Event Type  \
0  192.168.1.5      10.0.0.20    22  Failed Login Attempt   

                        Reason/Analysis                       Remediation  
0  Brute-force attack attempt detected.  Block suspicious IP, enable MFA.  


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss


API request failed with status code 404
Analysis for Row 1: {'error': 'API request failed with status code 404'}


In [50]:
# Assuming you want to store this information in variables or a data structure:

source_ip = "192.168.1.5"
destination_ip = "10.0.0.20"
port = 22
event_type = "Failed Login Attempt"


# Alternatively, if you want to print this out with better formatting, you can do:

print(f"Source IP: {source_ip}, Destination IP: {destination_ip}, Port: {port}, Event Type: {event_type}.")

SyntaxError: invalid syntax (<ipython-input-50-adbc930d1c6a>, line 1)